In [444]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [445]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [446]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [447]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [448]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
697,25/11/2020 20:47:48,35.0,Male,Munich,DevOps,17,5,Lead,Kubrrnetes,"R, AWS, Google Cloud, Kubernetes, Docker",72000.0,8000,30,Full-time employee,Unlimited contract,English,1000+,Consulting / Agency
511,25/11/2020 09:45:51,33.0,Male,Berlin,Software Engineer,12,5,Lead,Go,"Go, AWS, Docker",120000.0,50000,30,Full-time employee,Unlimited contract,English,101-1000,Product
458,24/11/2020 22:46:49,28.0,Male,Kyiv,Mobile Developer,5,0,Senior,Swift,"Swift, Google Cloud",45000.0,NaN,20,Full-time employee,Unlimited contract,English,11-50,Product
294,24/11/2020 15:37:24,25.0,Male,Stuttgart,Backend Developer,7,1,Senior,PHP,"Python, PHP, Javascript / Typescript, SQL",60000.0,NaN,26,Full-time employee,Unlimited contract,English,11-50,Startup
588,25/11/2020 13:51:32,32.0,Male,Hamburg,Product Manager,7,2,Senior,NaN,SQL,85000.0,5000,27,Full-time employee,Unlimited contract,English,101-1000,Product
608,25/11/2020 14:27:14,29.0,Male,Berlin,Backend Developer,5,1,Senior,Java,"Java / Scala, SQL, AWS, Kubernetes, Docker",70000.0,0,27,Full-time employee,Unlimited contract,English,1000+,Product
1033,01/12/2020 14:58:24,36.0,Male,Berlin,Software Engineer,10,10,Principal,Java & Distributed Systems Stuff,"Python, Javascript / Typescript, Java / Scala,...",100000.0,35000,28,Full-time employee,Unlimited contract,English,1000+,Product
242,24/11/2020 14:03:04,29.0,Female,Stuttgart,QA Engineer,6,2,Middle,JS,NaN,54000.0,NaN,26,Full-time employee,Unlimited contract,English,101-1000,Product
833,27/11/2020 12:42:00,39.0,Male,Munich,Software Engineer,8,7,Senior,Python,C/C++,130000.0,50000,30,Full-time employee,Unlimited contract,English,1000+,Product
91,24/11/2020 11:46:48,30.0,Male,Munich,Hardware Engineer,5,5,Senior,VHDL,Python,75000.0,10000,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [449]:
# Remove rows with NA position 
na_rows = df[df[df.columns[4]].isna()]
df.drop(na_rows.index, inplace=True)

# Get median age 
median_age = df['Age'].median()

# Fill NA in age with median age 
df['Age'].fillna(median_age, inplace=True)

# Fill NA in gender with other 
df['Gender'].fillna('Other', inplace=True)

# Fill NA in 'Years of experience in Germany' with 0
df['Years of experience in Germany'].fillna(0, inplace=True)
df.loc[df['Total years of experience'].isnull(), 'Total years of experience'] = df['Years of experience in Germany']

# Fill empty 'Your main technology / programming language' with 'Other'
df['Your main technology / programming language'].fillna('Other', inplace=True)

# Fill empty 'Other technologies/programming languages you use often' with 'None'
df['Other technologies/programming languages you use often'].fillna('None', inplace=True)

# Fill empty 'Yearly bonus + stocks in EUR' with 0
df['Yearly bonus + stocks in EUR'].fillna(0, inplace=True)

# Fill empty 'Number of Vacation days' with the mode 
mode_number_of_vacation = float(df['Number of vacation days'].mode())
df['Number of vacation days'].fillna(mode_number_of_vacation, inplace=True)

# Fill empty 'Seniority level' with 'Other'
df['Seniority level'].fillna('Other', inplace=True)

# Replace the low frequency with "Other"
def replace_low_freq(d, col, threshold=10, replacement='Other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp


# Replace low frequency values with "Other" for "Your main technology / programming language"
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")

# Replace specific values in "Your main technology / programming language"
df["Your main technology / programming language"].replace({"Javascript": "JavaScript", "Python ":"Python"}, inplace=True)

# Fill NA values in 'Employment status' with 'Other'
df['Employment status'].fillna('Other', inplace=True)

# Replace written mistake values in 'Employment status'
df['Employment status'].replace({'working student':'Working Student',
                                 'Werkstudent':'Working Student',
                                 'Full-time position, part-time position, & self-employed (freelancing, tutoring)':'Full-time employee',
                                 "full-time, but 32 hours per week (it was my request, I'm a student)":'Full-time employee',
                                 'Intern':'Working Student'}, inplace=True)

# Replace low frequency values with "Other" for "Employment status"
df["Employment status"] = replace_low_freq(df, "Employment status", 10, "Other")

# Fill NA values in 'Сontract duration' with 'Other'
df['Сontract duration'].fillna('Other', inplace=True)

# Replace low frequency values with "Other" for "Сontract duration"
df['Сontract duration'] = replace_low_freq(df, "Сontract duration", 10, "Other")

# Fill NA values in 'Main language at work' with 'Other'
df['Main language at work'].fillna('Other', inplace=True)

# Replace low frequency values with "Other" for "Main language at work"
df['Main language at work'] = replace_low_freq(df, 'Main language at work', 10, "Other")

# Fill NA values in 'Company size' with 'Other'
df['Company size'].fillna('Other', inplace=True)

# Fill NA values in 'Company type' with 'Other'
df['Company type'].fillna('Other', inplace=True)

# Replace low frequency values with "Other" for "Company type"
df['Company type'] = replace_low_freq(df, 'Company type', 10, "Other")

# Drop the 'Timestamp' column
df.drop(columns=['Timestamp'], inplace=True)

# Convert "Yearly bonus + stocks in EUR" to numeric, fill NA values with 0
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], errors="coerce")
df['Yearly bonus + stocks in EUR'].fillna(0, inplace=True)

# Calculate 'target' by adding "Yearly brutto salary (without bonus and stocks) in EUR" and "Yearly bonus + stocks in EUR"
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"]


df.isnull().sum()






Age                                                       0
Gender                                                    0
City                                                      0
Position                                                  0
Total years of experience                                 0
Years of experience in Germany                            0
Seniority level                                           0
Your main technology / programming language               0
Other technologies/programming languages you use often    0
Yearly brutto salary (without bonus and stocks) in EUR    0
Yearly bonus + stocks in EUR                              0
Number of vacation days                                   0
Employment status                                         0
Сontract duration                                         0
Main language at work                                     0
Company size                                              0
Company type                            

In [464]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error


# Assuming df is your DataFrame
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"].astype(float)

# Perform OH encoding on categorical columns
df_encoded = pd.get_dummies(df)

# Splitting data into X and y
X = df_encoded.drop(columns=["target"])
y = df_encoded["target"]

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
estimator = RandomForestRegressor(n_jobs=-1)

# Creating pipeline without feature selection
pipe_without_fs = Pipeline(steps=[("scaler", scaler), ("forest", estimator)])

# Creating pipeline with feature selection
pipe_with_fs = Pipeline(steps=[("scaler", scaler), ("feature_selection", SelectFromModel(estimator)), ("forest", estimator)])

# Defining parameters for grid search
params = {
    'forest__max_depth': [5, 6, 7, 8, 9, 10],
    "forest__n_estimators": [50, 75, 100, 125, 150],
    "forest__min_samples_leaf": [1, 2, 4],
    "forest__min_samples_split": [2, 3, 4, 5]
}
# Initializing GridSearchCV without feature selection
clf_without_fs = GridSearchCV(pipe_without_fs, param_grid=params, cv=5, n_jobs=-1)

# Fitting the model without feature selection
clf_without_fs.fit(X_train, y_train)

# Getting the best estimator without feature selection
best_estimator_without_fs = clf_without_fs.best_estimator_

# Fit the best model to get feature importance
best_estimator_without_fs.fit(X_train, y_train)

# Getting feature importances
feature_importances = best_estimator_without_fs.named_steps['forest'].feature_importances_

# Getting the indices of features selected
selected_indices = [i for i in range(len(feature_importances)) if feature_importances[i] > 0]

# Getting the list of selected features
selected_features = X.columns[selected_indices]

# Fitting the model with feature selection
clf_with_fs = GridSearchCV(pipe_with_fs, param_grid=params, cv=5, n_jobs=-1)

# Fitting the model with feature selection
clf_with_fs.fit(X_train, y_train)

# Getting the best estimator with feature selection
best_estimator_with_fs = clf_with_fs.best_estimator_

# Getting the predictions for train and test sets without feature selection
y_train_pred_without_fs = clf_without_fs.predict(X_train)
y_test_pred_without_fs = clf_without_fs.predict(X_test)

# Getting the predictions for train and test sets with feature selection
y_train_pred_with_fs = clf_with_fs.predict(X_train)
y_test_pred_with_fs = clf_with_fs.predict(X_test)

# Compute R2 and RMSE without feature selection
train_r2_without_fs = r2_score(y_train, y_train_pred_without_fs)
test_r2_without_fs = r2_score(y_test, y_test_pred_without_fs)
train_rmse_without_fs = mean_squared_error(y_train, y_train_pred_without_fs, squared=False)
test_rmse_without_fs = mean_squared_error(y_test, y_test_pred_without_fs, squared=False)

# Compute R2 and RMSE with feature selection
train_r2_with_fs = r2_score(y_train, y_train_pred_with_fs)
test_r2_with_fs = r2_score(y_test, y_test_pred_with_fs)
train_rmse_with_fs = mean_squared_error(y_train, y_train_pred_with_fs, squared=False)
test_rmse_with_fs = mean_squared_error(y_test, y_test_pred_with_fs, squared=False)

print("Results with Feature Selection:")
print("Train R2 Score:", train_r2_with_fs)
print("Test R2 Score:", test_r2_with_fs)
print("Train RMSE:", train_rmse_with_fs)
print("Test RMSE:", test_rmse_with_fs)

print("\nResults without Feature Selection:")
print("Train R2 Score:", train_r2_without_fs)
print("Test R2 Score:", test_r2_without_fs)
print("Train RMSE:", train_rmse_without_fs)
print("Test RMSE:", test_rmse_without_fs)

print("\nRemoved Features:")
print(selected_features)




Results with Feature Selection:
Train R2 Score: 0.8234704143346034
Test R2 Score: 0.9896135671280796
Train RMSE: 73147439.73879525
Test RMSE: 4073.792233217301

Results without Feature Selection:
Train R2 Score: 0.8630005208482874
Test R2 Score: 0.9895454778528561
Train RMSE: 64439155.013465755
Test RMSE: 4087.1234917696192

Removed Features:
Index(['Age', 'Yearly brutto salary (without bonus and stocks) in EUR',
       'Yearly bonus + stocks in EUR', 'Gender_Female', 'Gender_Male',
       'Gender_Other', 'City_Amsterdam', 'City_Berlin', 'City_Cologne',
       'City_Cupertino',
       ...
       'Main language at work_Russian', 'Company size_1000+',
       'Company size_101-1000', 'Company size_11-50', 'Company size_51-100',
       'Company size_up to 10', 'Company type_Consulting / Agency',
       'Company type_Other', 'Company type_Product', 'Company type_Startup'],
      dtype='object', length=201)


# Answers and Explainations

## Model Accuracy Comparison

- **Without Feature Selection:**
  - Train R2 Score: 0.8630005208482874
  - Test R2 Score: 0.9895454778528561
  - Train RMSE: 64439155.013465755
  - Test RMSE: 4087.1234917696192

- **With Feature Selection:**
  - Train R2 Score: 0.8234704143346034
  - Test R2 Score: 0.9896135671280796
  - Train RMSE: 73147439.73879525
  - Test RMSE: 4073.792233217301

## Feature Selection Activities

- **Approach 1:**
  - Method: Drop features with low frequency
  - Result: 201 features were removed.

## Hyperparameter Changes and Grid Search Improvements

- **Hyperparameter Changes:**
  - Parameters Modified: 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50
  - Reasoning: These hyperparameters were chosen based on domain knowledge and experimentation to achieve optimal model performance.
  - Results: The model achieved a slight improvement in performance after hyperparameter tuning.

- **Grid Search Improvements:**
  - Changes Made: Extended the range of values for 'max_depth', 'min_samples_leaf', 'min_samples_split', and 'n_estimators'.
  - Reasoning: To explore a wider range of hyperparameters and potentially discover better combinations for improved model performance.
  - Results: The grid search resulted in finding the best parameters that yielded a higher R2 score and lower RMSE compared to the initial hyperparameters.

## Overall Summary

- **Summary of Results:**
  - The best parameters obtained from the grid search are: 
    - 'max_depth': 9
    - 'min_samples_leaf': 1
    - 'min_samples_split': 5
    - 'n_estimators': 75
  - The best estimator achieved an R2 Score of 0.9896 and a Root Mean Squared Error (RMSE) of 4073.79.
  - Feature selection resulted in a slight decrease in model performance, as evidenced by the lower R2 score and higher RMSE compared to the model without feature selection. However, the impact on model performance was minimal.